In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix


In [3]:
pd.set_option('display.max_columns', None)
sns.set(style="darkgrid")
data_file = '/content/drive/MyDrive/Wednesday-workingHours.pcap_ISCX.csv'
df = pd.read_csv(data_file, low_memory=False, on_bad_lines='skip')  # skip bad rows
df.columns = df.columns.str.strip() # Clean Column Name, remove extra spaces from Names etc.
print("✅ Data loaded. Shape:", df.shape)

✅ Data loaded. Shape: (1118686, 79)


In [4]:
print("📋 Total Columns:", len(df.columns))
print("🧾 Column Names:")
print(df.columns.tolist())

📋 Total Columns: 79
🧾 Column Names:
['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count

In [5]:
print("\n🔍 Unique Labels in 'Label' Column:")
print(df['Label'].unique())


🔍 Unique Labels in 'Label' Column:
['BENIGN' nan 'DoS Hulk' 'DoS slowloris' 'DoS GoldenEye'
 'DoS Slowhttptest' 'Heartbleed']


In [6]:
# Label counts
print("\n📊Label Distribution:")
print(df['Label'].value_counts())


📊Label Distribution:
Label
BENIGN              804930
DoS Hulk            281753
DoS GoldenEye        20580
DoS slowloris         5795
DoS Slowhttptest      5497
Heartbleed              22
Name: count, dtype: int64


In [7]:
# Step 3: Handle Infinite and Missing Values
# --------------------------
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)

In [8]:
from sklearn.preprocessing import LabelEncoder
# Binary classification: Normal vs Attack
df['Label'] = df['Label'].astype(str).str.strip()
df['Label'] = df['Label'].apply(lambda x: 'Normal' if x.upper() == 'BENIGN' else 'Attack')

# Encode as 0/1
label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])  # Attack=0 or 1
print("Label Mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

Label Mapping: {'Attack': np.int64(0), 'Normal': np.int64(1)}


In [9]:
print("\n🔍 Unique Labels in 'Label' Column:")
print(df['Label'].unique())
print("\n📊Label Distribution:")
print(df['Label'].value_counts())


🔍 Unique Labels in 'Label' Column:
[1 0]

📊Label Distribution:
Label
1    804930
0    313756
Name: count, dtype: int64


In [10]:
# Step 5: Separate Features and Labels
# --------------------------
X = df.drop('Label', axis=1)
y = df['Label']

In [11]:
# Ensure all features are numeric
X = X.select_dtypes(include=[np.number])

In [12]:
# Step 6: Scale the Features
# --------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [13]:
# Step 7: Train-Test Split
# --------------------------
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

In [14]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

# STEP 1: Reshape input for CNN: (samples, features, 1)
X_train_cnn = X_train.reshape(-1, X_train.shape[1], 1)
X_test_cnn = X_test.reshape(-1, X_test.shape[1], 1)

# STEP 2: Define the CNN model
model = Sequential()

model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(1, activation='sigmoid'))  # Binary classification

# STEP 3: Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# STEP 4: Train the model
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(X_train_cnn, y_train,
                    epochs=10,
                    batch_size=256,
                    validation_split=0.2,
                    callbacks=[early_stop],
                    verbose=2)

# STEP 5: Save the model for later use for another dataset

model.save("cnn_cicids_model.h5")

# STEP 5: Evaluate on test data
loss, accuracy = model.evaluate(X_test_cnn, y_test, verbose=0)
print(f"\nTest Accuracy: {accuracy:.4f}")

# STEP 6: Predictions and performance report
y_pred_prob = model.predict(X_test_cnn)
y_pred = (y_pred_prob > 0.5).astype(int)

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
2448/2448 - 97s - 40ms/step - accuracy: 0.9387 - loss: 0.1354 - val_accuracy: 0.9364 - val_loss: 0.1131
Epoch 2/10
2448/2448 - 92s - 37ms/step - accuracy: 0.9555 - loss: 0.1005 - val_accuracy: 0.9034 - val_loss: 0.1680
Epoch 3/10
2448/2448 - 93s - 38ms/step - accuracy: 0.9649 - loss: 0.0824 - val_accuracy: 0.9035 - val_loss: 0.2915
Epoch 4/10
2448/2448 - 92s - 38ms/step - accuracy: 0.9672 - loss: 0.0772 - val_accuracy: 0.9007 - val_loss: 0.6844



Test Accuracy: 0.9359
10488/10488 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step

Classification Report:

              precision    recall  f1-score   support

           0       0.82      0.99      0.90     94127
           1       1.00      0.91      0.95    241479

    accuracy                           0.94    335606
   macro avg       0.91      0.95      0.93    335606
weighted avg       0.95      0.94      0.94    335606

Confusion Matrix:
[[ 93572    555]
 [ 20942 220537]]
